<a href="https://colab.research.google.com/github/aderobaki/Data-Science-Projs/blob/workflows/Copy_of_Sales_Data_prediction_using_Time_series-rev1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lets first discuss what we are given and what we have to predict.
About our dataset :

We have in our training data :- 
1. date - every date of items sold
2. date_block_num - this number given to every month
3. shop_id - unique number of every shop
4. item_id - unique number of every item
5. item_price - price of every item
6. item_cnt_day - number of items sold on a particular day 

We have in our testing data :- 
1. ID - unique for every (shop_id,item_id) pair.
2. shop_id - unique number of every shop
3. item_id - unique number of every item

Now what we have to predict ?
we have to predict how many items of a type from each shop  will be sold in a whole month.
Our submission should have ID and item_cnt_month columns.

What is our approach?
our approach will be simple.
Our features will be number of items sold in month from a shop excluding last month data because that will our labels, that we help our model learn to predict next sequence. And for testing will use number of items sold in month from a shop excluding first month like this dimension of our data remains same. Our model will predict the next sequence and that we will be our results. This is pretty simple approach but its good for start. Please try some different approaches also. 
And please let me know if I did something wrong. If you like it please vote it up.


### **File descriptions**
- sales_train.csv - the training set. Daily historical data from January 2013 to October 2015.
- test.csv - the test set. You need to forecast the sales for these shops and products for November 2015.
- sample_submission.csv - a sample submission file in the correct format.
- items.csv - supplemental information about the items/products.
- item_categories.csv - supplemental information about the items categories.
- shops.csv- supplemental information about the shops.

In [2]:
!pip install pmdarima

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.8 MB 14.7 MB/s 
     |████████████████████████████████| 9.8 MB 63.3 MB/s 
  Attempting uninstall: statsmodels
    Found existing installation: statsmodels 0.12.2
    Uninstalling statsmodels-0.12.2:
      Successfully uninstalled statsmodels-0.12.2


In [16]:
import pandas as pd
import numpy as np
import datetime
import os
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import scipy

# To tune model, get different metric scores, and split data
from sklearn.metrics import  precision_recall_curve, roc_auc_score, confusion_matrix, accuracy_score, recall_score, precision_score, f1_score,auc, roc_curve, plot_confusion_matrix
from sklearn import metrics

from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score


from sklearn.model_selection import GridSearchCV
from pmdarima.arima import auto_arima
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

In [6]:
df_train= pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Sales Data Prediction/sales_train.csv")
df_test= pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Sales Data Prediction/test.csv")

In [7]:
df_train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [9]:
df_test.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [8]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2935849 entries, 0 to 2935848
Data columns (total 6 columns):
 #   Column          Dtype  
---  ------          -----  
 0   date            object 
 1   date_block_num  int64  
 2   shop_id         int64  
 3   item_id         int64  
 4   item_price      float64
 5   item_cnt_day    float64
dtypes: float64(2), int64(3), object(1)
memory usage: 134.4+ MB


In [10]:
df_train.sample(10, random_state=42)

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
2472142,06.02.2015,25,31,11208,699.0,1.0
1044690,21.11.2013,10,31,19525,149.0,1.0
2367559,03.01.2015,24,42,16169,299.0,1.0
1793501,18.07.2014,18,42,2737,199.0,1.0
1585639,13.04.2014,15,2,15229,1199.0,1.0
1229976,27.12.2013,11,57,2269,799.0,1.0
183439,08.02.2013,1,31,18151,149.0,1.0
2343443,08.01.2015,24,27,6184,699.0,1.0
485890,09.05.2013,4,54,4908,899.0,1.0
431646,27.04.2013,3,42,2447,1999.0,1.0


In [11]:
df_train.describe()

,date_block_num,shop_id,item_id,item_price,item_cnt_day
count,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06
mean,1.456991e+01,3.300173e+01,1.019723e+04,8.908532e+02,1.242641e+00
std,9.422988e+00,1.622697e+01,6.324297e+03,1.729800e+03,2.618834e+00
min,0.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+00,-2.200000e+01
25%,7.000000e+00,2.200000e+01,4.476000e+03,2.490000e+02,1.000000e+00
50%,1.400000e+01,3.100000e+01,9.343000e+03,3.990000e+02,1.000000e+00
75%,2.300000e+01,4.700000e+01,1.568400e+04,9.990000e+02,1.000000e+00
max,3.300000e+01,5.900000e+01,2.216900e+04,3.079800e+05,2.169000e+03


In [12]:
df_train.shape

(2935849, 6)

### Data Preprocessing

In [13]:
train_dataset = df_train.pivot_table(index=['shop_id', 'item_id'], values=['item_cnt_day'], columns=['date_block_num'], fill_value=0, aggfunc='sum')

In [18]:
train_dataset.head(3)

item_cnt_day                              ...                 \
date_block_num            0   1  2  3  4  5  6  7  8  9   ... 24 25 26 27 28   
shop_id item_id                                           ...                  
0       30                 0  31  0  0  0  0  0  0  0  0  ...  0  0  0  0  0   
        31                 0  11  0  0  0  0  0  0  0  0  ...  0  0  0  0  0   
        32                 6  10  0  0  0  0  0  0  0  0  ...  0  0  0  0  0   

                                
date_block_num  29 30 31 32 33  
shop_id item_id                 
0       30       0  0  0  0  0  
        31       0  0  0  0  0  
        32       0  0  0  0  0  

[3 rows x 34 columns]

In [ ]:
#convert negative values in item_price
df_train['item_price'] = abs(df_train['item_price'])
df_train['item_cnt_day'] = abs(df_train['item_cnt_day'])

In [ ]:
#change date from obj to datetime

df_train['date'] = df_train['date'].apply(lambda x:datetime.datetime.strptime(x, '%d.%m.%Y'))

In [ ]:
df_train['month'] = df_train['date'].dt.to_period('M').astype(str)


In [ ]:
df_train['month'] = pd.to_datetime(df_train['month'])
df_train.dtypes

In [ ]:
df_train["sale_revenue"]=df_train["item_cnt_day"] * df_train["item_price"] 

In [ ]:
df_train.sample(10, random_state=42)

In [ ]:
df_train.shape

### **Exploratory Data Analsyis**

In [ ]:
# General statistics
def stats(df,x):
    print(f"Variable: {x}")
    print(f"Type of variable: {df[x].dtype}")
    print(f"Total observations: {df[x].shape[0]}")
    detect_null_val = df[x].isnull().values.any()
    if detect_null_val:
        print(f"Missing values: {df[x].isnull().sum()} ({(df[x].isnull().sum() / df[x].isnull().shape[0] *100).round(2)}%)")
    else:
        print(f"Missing values? {df[x].isnull().values.any()}")
    print(f"Unique values: {df[x].nunique()}")
    if df[x].dtype != "O":
        print(f"Min: {int(df[x].min())}")
        print(f"25%: {int(df[x].quantile(q=[.25]).iloc[-1])}")
        print(f"Median: {int(df[x].median())}")
        print(f"75%: {int(df[x].quantile(q=[.75]).iloc[-1])}")
        print(f"Max: {int(df[x].max())}")
        print(f"Mean: {df[x].mean()}")
        print(f"Std dev: {df[x].std()}")
        print(f"Variance: {df[x].var()}")
        print(f"Skewness: {scipy.stats.skew(df[x])}")
        print(f"Kurtosis: {scipy.stats.kurtosis(df[x])}")
        print("")
        
        # Percentiles 1%, 5%, 95% and 99%
        print("Percentiles 1%, 5%, 95%, 99%")
        display(df[x].quantile(q=[.01, .05, .95, .99]))
        print("")
    else:
        print(f"List of unique values: {df[x].unique()}")

# Histogram
def hist(x):
    plt.hist(df[x], bins=25)
    plt.title(x, fontsize=10, loc="right")
    plt.xlabel('Relative frequency')
    plt.ylabel('Absolute frequency')
    plt.show()


In [ ]:
stats(df_train,"item_cnt_day")

In [ ]:
 #lets see the revenue per shop
 store = df_train.groupby('shop_id')['sale_revenue'].sum().reset_index()
  
fig, ax = plt.subplots(figsize=(16,4))
sns.barplot(store.shop_id, store.sale_revenue, color='mediumblue')
ax.set(xlabel = "Store ID",ylabel = "Sale Revenue", title = "Total Sales Per Store")
    
sns.despine()

In [ ]:
plt.figure(figsize = (9, 6))
plt.grid()
plt.plot(df_train['item_cnt_day'])
plt.title('Daily Sales of items')
plt.xlabel('Time')
plt.ylabel('Sales count')
plt.show()

In [ ]:
df_train[['shop_id','item_id','item_price','item_cnt_day']].corr()

### **Time Series Analysis**

In [ ]:
ts=df_train.groupby(["date_block_num"])["item_cnt_day"].sum()
ts.astype('float')

plt.figure(figsize=(16,8))
plt.grid()
plt.title('Total Sales of the company')
plt.xlabel('Time')
plt.ylabel('Sales')
plt.plot(ts);

In [ ]:
plt.figure(figsize=(16,6))
plt.grid()
plt.plot(ts.rolling(window=12,center=False).mean(),label='Rolling Mean');
plt.plot(ts.rolling(window=12,center=False).std(),label='Rolling sd');
plt.legend();

In [ ]:
data =df_train.groupby(["month"]).agg({'item_cnt_day':"sum"})
data
data.shape

In [ ]:
data["month"] =data.index
data

In [ ]:
data.drop(["month"], axis =1, inplace=True)
data

In [ ]:
data["item_cnt_mnt"] = data["item_cnt_day"]
data.head()

In [ ]:
data.drop(["item_cnt_day"], axis =1, inplace=True)

In [ ]:
plt.figure(figsize = (16,8))
plt.grid()
plt.plot(data['item_cnt_mnt'])
plt.title('Monthly Sales of items')
plt.xlabel('Time')
plt.ylabel('Sales count')
plt.show()

In [ ]:
pd.plotting.autocorrelation_plot(data['item_cnt_mnt'])
print('Autocorrelation =', round(data['item_cnt_mnt'].autocorr(), 4))

In [ ]:
plot_acf(data['item_cnt_mnt'])
plt.grid()
plot_pacf(data['item_cnt_mnt'])
plt.grid()
plt.show()

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller

In [ ]:
#Additive model
seasonal_decompose(data['item_cnt_mnt'], model = 'additive').plot().set_size_inches(10, 8) #default

In [ ]:
#Additive model
seasonal_decompose(data['item_cnt_mnt'], model = 'multiplicative').plot().set_size_inches(10, 8)

In [ ]:
#Stationarity test: Two tests for checking the stationarity of a time series are used, namely the ADF test and the KPSS test.
def adfTester(timeseries):
    adfTest = adfuller(timeseries, autolag='AIC')
    output = pd.Series(adfTest[0:4], index = ['ADF Statistic', 'p-value', 'Lags', 'Number of Observations'])
    for key, value in adfTest[4].items():
        output["Critical Value (%s)" %key] = value
    
    return print(output)

In [ ]:
adfTester(data['item_cnt_mnt'])

p-value for the DF test is well within 5 %. Hence we can assume Stationarity of the series.

In [ ]:
#Grid search for parameters
def order_parameters(training_data):
    
    search_params = auto_arima(training_data, start_p = 0, start_q = 0, m = 12, seasonal = True, test = "adf", d = None, trace = True, alpha = 0.05, information_criterion = 'aic', suppress_warnings = True, 
                    stepwise = True)
 
    print("AIC = ", round(search_params.aic(), 2))
                  
    return search_params

In [ ]:
model = order_parameters(data['item_cnt_mnt'])

In [ ]:
print(model.summary())

In [ ]:
prediction, confint = model.predict(n_periods = 6, return_conf_int = True) #95% CI default
df_confint = pd.DataFrame(confint)
print(confint.round(2))
print(prediction.round(2))

In [ ]:
period_index = pd.period_range(start = data.index[-1], periods = 6, freq='M')

df_predict = pd.DataFrame({'Predicted item_cnt_mnt': prediction.round(2)}, index = period_index)
#df_predict['month'] = df_predict.index
print(df_predict)

In [ ]:
plt.figure(figsize = (10, 6))
plt.plot(data['item_cnt_mnt'], label = 'Actuals')
plt.plot(df_predict.to_timestamp(), color = 'orange', label = 'Predicted')
plt.fill_between(period_index.to_timestamp(), df_confint[0], df_confint[1], color = 'grey', alpha = 0.25, label = 'Confidence Interval')
plt.legend(loc = 'lower left')
plt.title('Time-series Forecasting (SARIMA)')
plt.grid()
plt.show()

In [ ]:
train_df_tuple = df_train.groupby(['shop_id', 'item_id'])['date', 'item_cnt_day'].agg({'item_cnt_day':'sum'})
train_df_tuple = train_df_tuple.reset_index()
print(train_df_tuple)

In [ ]:
df_test['item_cnt_month'] = (prediction[0].round(2)*len(df_test)/len(train_df_tuple))/len(df_test)
#df_test['item_cnt_month'] = prediction[0]*len(df_test)/len(train_df_tuple)
submission  = df_test.drop(['shop_id', 'item_id'], axis = 1)
print(submission)

In [ ]:
submission.to_csv('submission.csv', index = False)